# 1. Initial Setup

In this first section, we'll simply do some of the initial stuff before actually getting to work with data. This will include package imports, loading the datasets and, if the need arises, other configurations.

In [1]:
#Import packages
import os
import pandas as pd

import requests
from bs4 import BeautifulSoup

In [2]:
#Load data
INPUT_PATH = "/kaggle/input/steam-games-dataset/"
df = pd.read_json(INPUT_PATH + "games.json", orient="index")
df.index.rename("app_id", inplace=True)

# 2. Early Exploration

We now start actually looking into the data. The goal here is to have a sense of what we are working with. The main outcomes from this step are getting more familiar with the dataset and also possibly excluding data that clearly will not be used in the following analyses.

In [3]:
#Get a sense of what the data looks like
df.head()

,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,header_image,...,score_rank,positive,negative,estimated_owners,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags
app_id,,,,,,,,,,,,,,,,,,,,,
20200,Galactic Bowling,"Oct 21, 2008",0,19.99,0,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,6,11,0 - 20000,0,0,0,0,0,"{'Indie': 22, 'Casual': 21, 'Sports': 21, 'Bow..."
655370,Train Bandit,"Oct 12, 2017",0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,53,5,0 - 20000,0,0,0,0,0,"{'Indie': 109, 'Action': 103, 'Pixel Graphics'..."
1732930,Jolt Project,"Nov 17, 2021",0,4.99,0,Jolt Project: The army now has a new robotics ...,Jolt Project: The army now has a new robotics ...,"Shoot vehicles, blow enemies with a special at...",,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,0,0,0 - 20000,0,0,0,0,0,[]
1355720,Henosis™,"Jul 23, 2020",0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,3,0,0 - 20000,0,0,0,0,0,"{'2D Platformer': 161, 'Atmospheric': 154, 'Su..."
1139950,Two Weeks in Painland,"Feb 3, 2020",0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,ABOUT THE GAME Play as a hacker who has arrang...,Two Weeks in Painland is a story-driven game a...,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,,50,8,0 - 20000,0,0,0,0,0,"{'Indie': 42, 'Adventure': 41, 'Nudity': 22, '..."


In [4]:
#Ensure the index (app_id) is the unique key
df.index.value_counts()

app_id
20200      1
2016110    1
2193610    1
1997040    1
1965920    1
          ..
380        1
1475740    1
1021680    1
816250     1
3054200    1
Name: count, Length: 97410, dtype: int64

In [5]:
#Look into the data columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97410 entries, 20200 to 3054200
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      97410 non-null  object 
 1   release_date              97410 non-null  object 
 2   required_age              97410 non-null  int64  
 3   price                     97410 non-null  float64
 4   dlc_count                 97410 non-null  int64  
 5   detailed_description      97410 non-null  object 
 6   about_the_game            97410 non-null  object 
 7   short_description         97410 non-null  object 
 8   reviews                   97410 non-null  object 
 9   header_image              97410 non-null  object 
 10  website                   97410 non-null  object 
 11  support_url               97410 non-null  object 
 12  support_email             97410 non-null  object 
 13  windows                   97410 non-null  bool   
 14  mac  

From this step, it looks as though all fields hold valid values for the entire dataset. While a positive surprise, we should try and look deeper into some specific fields. At this moment, we want to understand if columns such as `notes` hold any insightful data.

## 2.1. Early Exclusion Candidates

Here, we are going to attempt to perform an initial filter on fields we might deem irrelevant for the analysis. As we have explored the data much at this point, the selected fields for investigation are based on the sample data we already saw and on the perceived information they hold based on their names.

For each field, we might perform different evaluations. What you are going to see here is the result of some behind the curtain analysis.

In [6]:
#header_image
df.header_image.value_counts()

header_image
https://cdn.akamai.steamstatic.com/steam/apps/750920/header.jpg?t=1644931091                          20
https://cdn.akamai.steamstatic.com/steam/apps/24010/header.jpg?t=1658409119                            5
https://cdn.akamai.steamstatic.com/steam/apps/359550/header.jpg?t=1655223333                           5
https://cdn.akamai.steamstatic.com/steam/apps/37960/header.jpg?t=1447352428                            4
https://cdn.akamai.steamstatic.com/steam/apps/2195250/header.jpg?t=1701266064                          4
                                                                                                      ..
https://cdn.akamai.steamstatic.com/steam/apps/29700/header.jpg?t=1598612361                            1
https://cdn.akamai.steamstatic.com/steam/apps/804700/header.jpg?t=1647259710                           1
https://cdn.akamai.steamstatic.com/steam/apps/365580/header.jpg?t=1447373140                           1
https://cdn.akamai.steamstatic.com/steam/a

In [7]:
#website
df.website.value_counts()

website
                                                 54673
https://www.facebook.com/8FloorGames/              274
http://www.exosyphen.com                           204
https://www.choiceofgames.com/                     153
https://steamcommunity.com/groups/alawargames       92
                                                 ...  
https://vasonastudios.com/                           1
https://www.facebook.com/MistSurvival/               1
http://www.xperienciavirtual.es/                     1
https://tendaystowar.zerobyter.net/                  1
https://www.dreamengineproductions.com/              1
Name: count, Length: 34763, dtype: int64

In [8]:
#support_url
df.support_url.value_counts()

support_url
                                              51463
https://www.facebook.com/8FloorGames            298
https://bigfishgames.zendesk.com/hc/en-us/      260
https://bigfishgames.custhelp.com/app/home      223
https://www.choiceofgames.com/contact-us/       204
                                              ...  
http://support.immersedgames.com                  1
https://discord.gg/QnnUMCM2w3                     1
touwaku.com                                       1
https://twitter.com/TD12734                       1
https://www.dreamengineproductions.com/           1
Name: count, Length: 30083, dtype: int64

In [9]:
#support_email and support_url
aux = df[["support_url", "support_email"]]
aux["has_support"] = (aux != '').max(axis=1)
aux.has_support.value_counts()

<ipython-input-9-e99ee1aebd4f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux["has_support"] = (aux != '').max(axis=1)


has_support
True     91586
False     5824
Name: count, dtype: int64

In [10]:
#achievements
df.achievements.value_counts().sort_index()

achievements
0       50189
1        1058
2         594
3         801
4        1011
        ...  
4997        1
4999        1
5000       59
5394        1
9821        1
Name: count, Length: 434, dtype: int64

In [11]:
#recommendations
df.recommendations.value_counts()

recommendations
0        83062
116         69
105         68
106         67
109         66
         ...  
9528         1
1459         1
10535        1
3539         1
6520         1
Name: count, Length: 4094, dtype: int64

In [12]:
#notes
df.notes.value_counts()

notes
                                                                                                                                                                                                                           81936
This Game may contain content not appropriate for all ages, or may not be appropriate for viewing at work: Frequent Violence or Gore, General Mature Content                                                                 122
Nakedness. All characters appearing in this game are over 18 years of age.                                                                                                                                                   115
-Nudity or sexual content -Not suitable for watching at work -Not suitable for all ages -All characters in the game are over 18 years old                                                                                     67
This Game may contain content not appropriate for all ages, or may not be appropriate for view

In [13]:
#packages
df.packages.head()

app_id
20200      [{'title': 'Buy Galactic Bowling', 'descriptio...
655370     [{'title': 'Buy Train Bandit', 'description': ...
1732930    [{'title': 'Buy Jolt Project', 'description': ...
1355720    [{'title': 'Buy Henosis™', 'description': '', ...
1139950                                                   []
Name: packages, dtype: object

In [14]:
df.packages.head(10).values

array([list([{'title': 'Buy Galactic Bowling', 'description': '', 'subs': [{'text': 'Galactic Bowling - $19.99', 'description': '', 'price': 19.99}]}]),
       list([{'title': 'Buy Train Bandit', 'description': '', 'subs': [{'text': 'Train Bandit - $0.99', 'description': '', 'price': 0.99}]}]),
       list([{'title': 'Buy Jolt Project', 'description': '', 'subs': [{'text': 'Jolt Project - $4.99', 'description': '', 'price': 4.99}]}]),
       list([{'title': 'Buy Henosis™', 'description': '', 'subs': [{'text': 'Henosis - $5.99', 'description': '', 'price': 5.99}]}]),
       list([]), list([]),
       list([{'title': 'Buy TD Worlds', 'description': '', 'subs': [{'text': 'TD Worlds - $10.99', 'description': '', 'price': 10.99}]}]),
       list([{'title': 'Buy Legend of Rome - The Wrath of Mars', 'description': '', 'subs': [{'text': 'Legend of Rome - The Wrath of Mars - $9.99', 'description': '', 'price': 9.99}]}]),
       list([{'title': 'Buy MazM: Jekyll and Hyde', 'description': '', 'su

In [15]:
#screenshots
df.screenshots.head()

app_id
20200      [https://cdn.akamai.steamstatic.com/steam/apps...
655370     [https://cdn.akamai.steamstatic.com/steam/apps...
1732930    [https://cdn.akamai.steamstatic.com/steam/apps...
1355720    [https://cdn.akamai.steamstatic.com/steam/apps...
1139950    [https://cdn.akamai.steamstatic.com/steam/apps...
Name: screenshots, dtype: object

In [16]:
#movies
df.movies.head()

app_id
20200      [http://cdn.akamai.steamstatic.com/steam/apps/...
655370     [http://cdn.akamai.steamstatic.com/steam/apps/...
1732930    [http://cdn.akamai.steamstatic.com/steam/apps/...
1355720    [http://cdn.akamai.steamstatic.com/steam/apps/...
1139950    [http://cdn.akamai.steamstatic.com/steam/apps/...
Name: movies, dtype: object

From a quick analysis of the columns above, we came if a few insights:
* None of the investigated fields should be removed at this moment. In one way or another, they present some potential for the upcoming analyses;
* **Some of the fields could use some transformation**. For instance, the `screenshots` column is not very useful as is. We could, however, count the amount of existing screenshot for a title and use it instead;
* The `header_image` distritibution showed some very interesting behaviour. Even though we validated the index coming from the data is unique, we see a **number of instances in which different games have the same header image**. While this is not necessarily an issue, the best course of action is to explore this further to ensure data quality.

Let's now take care of the highlighed points, starting with the data quality concerns.

## 2.2. Investigate Potential Duplicates

In [17]:
#Identify potential duplicates
duplicates = df.header_image.value_counts()
duplicates = duplicates[duplicates > 1]

duplicates_df = df.join(on="header_image", other=duplicates, how="inner").sort_values(by="header_image")

In [18]:
#Validate which field are equal and which are not across potential duplicates
for col in duplicates_df.columns:
    duplicates_df[col] = [str(val) for val in duplicates_df[col]]

duplicates_agg = duplicates_df.groupby(by="header_image").agg(
    {col: "nunique" for col in duplicates_df.columns}
)

duplicate_fields = []
for col in duplicates_agg.columns:
    duplicate_fields.append([col, sum(duplicates_agg[col] > 1)])

duplicate_fields = pd.DataFrame(duplicate_fields, columns=["field_name", "instances_with_distinct_values"])
duplicate_fields.sort_values(by="instances_with_distinct_values", ascending=False)

,field_name,instances_with_distinct_values
32,positive,67
33,negative,67
40,tags,66
37,median_playtime_forever,66
35,average_playtime_forever,66
39,peak_ccu,62
34,estimated_owners,51
19,recommendations,30
38,median_playtime_2weeks,22
36,average_playtime_2weeks,22


From the analysis above, we can draw a few conclusions:
* The major part of fields hold the exact same value across records with the same `header_image`;
* Some of these fields with no change are `name`, `screenshots`, `movies` and `detailed_description`. It's really hard to imagine completely distinct titles would share the same values for all those fields at the same time;
* Finally, we do have a smaller amount of fields for which is more common to have discrepant values. Some of them are `tags` and `medium_playtime_forever`.

While this analysis does point to these being duplicate records. We don't feel confident enough to act on it at this point. Another reason for this is we have no means of identifying the "correct" record, the one that should be kept. As some fields to present different values for the same potential duplicate instance, this choice could have repercussions later on.

What we did then was to actually visit the Steam pages for some of those games. Even though we don't have the actual URLs in the data, game pages are all structured like `https://store.steampowered.com/app/{app_id}`. As we do have the app ID as the dataset index, we have the opportunity to test a few of those cases. On the tested examples, we were able to see links for different `app_ids` were redirected to the same page. This is yet another strong indication these are indeed duplicate records.

Unfortunately, this does not addresses the issue of selecting the "right" row for each case. To do so, we are going to compare the provided app IDs in the data with the identifier found in the URL after being redirected.

In [19]:
#Identify the "main" app_id for duplicate cases
duplicates_df["redirect_url_app_id"] = [
    requests.get(url=f"https://store.steampowered.com/app/{app_id}", allow_redirects=True).url 
    for app_id in duplicates_df.index
]

duplicates_df["redirect_url_app_id"] = [
    int(url.replace("https://store.steampowered.com/app/", "").replace("/", "")) for url in duplicates_df["redirect_url_app_id"]
]

In [20]:
#Check matches between app_id and redirected_url_app_id
duplicates_df["is_match"] = duplicates_df.index == duplicates_df["redirect_url_app_id"]

duplicates_validation = duplicates_df.groupby(by="header_image").agg({
    "is_match": ["max", "sum"]
})
duplicates_validation.columns = ["has_match", "matches_count"]

duplicates_validation.value_counts()

has_match  matches_count
True       1                71
Name: count, dtype: int64

The results above show all 71 duplicate cases have only a single match between the app ID and the identifier coming from the redirected URL. It also shows there is only one match for each case. From these results, we can conclude this approach is valid for deduplicating records.